# Tweet Collection and Storing Them

* Insert tweets into db with some id, hashing its text and user


### FoxTV Ana Haber Bülteni Hashtag'leri

* 2 Nisan 2020 : #şüpheduymuyorum
* 3 Nisan 2020 : #duysesimi
* 4 Nisan 2020 : #
* 5 Nisan 2020 : #
* 6 Nisan 2020 : #neyinnesi (Erdoğan Açıklama 19:00)
* 7 Nisan 2020 : #biralkışda
* 8 Nisan 2020 : #hepsinitopla
* 9 Nisan 2020 : #kabuledilemez
* 10 Nisan 2020 : #neredenbulacağım

* 13 Nisan : #gittimgeldim
* 14 Nisan : #işaş
* 15 Nisan : #veresiyedefteri
* 16 Nisan : #çağrımvar
* 17 Nisan : #sesverin

* 20 Nisan : #çıldıracağım
* 21 Nisan : #nereyevarır
* 22 Nisan : #çocuklarınmutluluğu
* 23 Nisan : #halkınmeclisi
* 24 Nisan : #akıllarazarar

In [1]:
import tweepy
from pprint import pprint
from pymongo import MongoClient
from pymongo.errors import BulkWriteError
import json

In [2]:
client = MongoClient('mongodb://localhost:27017/')

db = client.covid_hack
tweets_db = db.tweets

tweets_db.find_one()

{'_id': ObjectId('5e8f04f3411c489a86a6e7a2'),
 'created_at': 'Thu Apr 09 10:10:40 +0000 2020',
 'id': 1248191593850568709,
 'id_str': '1248191593850568709',
 'full_text': 'RT @erdmozvrn: Şu Maaşım\nŞuda Kartım Yavrum..\n\n#Şüpheduymuyorum https://t.co/4hlD0Yc4yC',
 'truncated': False,
 'display_text_range': [0, 87],
 'entities': {'hashtags': [{'text': 'Şüpheduymuyorum', 'indices': [47, 63]}],
  'symbols': [],
  'user_mentions': [{'screen_name': 'erdmozvrn',
    'name': 'Erdem Özveren',
    'id': 2365427365,
    'id_str': '2365427365',
    'indices': [3, 13]}],
  'urls': [],
  'media': [{'id': 1245763183685570560,
    'id_str': '1245763183685570560',
    'indices': [64, 87],
    'media_url': 'http://pbs.twimg.com/ext_tw_video_thumb/1245763183685570560/pu/img/qyAgAd7g7ALW-ekf.jpg',
    'media_url_https': 'https://pbs.twimg.com/ext_tw_video_thumb/1245763183685570560/pu/img/qyAgAd7g7ALW-ekf.jpg',
    'url': 'https://t.co/4hlD0Yc4yC',
    'display_url': 'pic.twitter.com/4hlD0Yc4yC',
    'ex

In [3]:
file = open('keys.json')
key_dict = json.loads(file.read())
file.close()

In [4]:
auth = tweepy.OAuthHandler(key_dict['consumer_key'], key_dict['consumer_secret'])
auth.set_access_token(key_dict['access_token'], key_dict['access_token_secret'])

api = tweepy.API(auth)

In [5]:
def reshape_tweet(tweet, hashtag_id):
    tweet['hashtag_id'] = hashtag_id
    return tweet

In [32]:
searched_hashtags = ['#şüpheduymuyorum', '#duysesimi', '#biralkısda', '#neyinnesi', "#biralkışda", '#hepsinitopla', '#kabuledilemez', '#gittimgeldim', '#işaş', '#veresiyedefteri', '#çağrımvar', '#cağrımvar', '#sesverin', '#çıldıracağım', '#cıldıracagım', '#çıldıracagım', '#cıldıracağım', '#nereyevarır', '#çocuklarınmutluluğu', '#halkınmeclisi']

In [33]:
tweet_search_continue = True
check_all_for_dup = True
hashtag_id = '#akıllarazarar -filter:retweets'
max_id = None
request_number = 0

In [34]:
f = open('conflicts.txt', 'a+')

while tweet_search_continue:
    
    print("Request {}".format(request_number))
    
    request_number += 1
    
    tweets = api.search(q=hashtag_id, count=100, lang="tr", max_id=max_id, tweet_mode="extended")
    
    if len(tweets) == 0:
        print("No tweets fetched")
        break
    
    tweets = [tweet._json for tweet in tweets]

    print("Number of tweets {}".format(len(tweets)))
    
    tweets_filtered = []
    for tweet in tweets:
        
        # Check if it is a RT, if RT do no include
        if tweet['full_text'].startswith('RT'):
            continue
        
        no_conflict = True
        
        for hashtag in searched_hashtags:
            if hashtag in tweet['full_text']:
                print("Hashtag {} found in tweet {}".format(hashtag, tweet['id']))
                found = tweets_db.find_one({'id': tweet['id']})
                if found:
                    print("{}:{}:{}".format(hashtag_id, hashtag,tweet['id']), file=f)
                    no_conflict = False
        if no_conflict:
            tweets_filtered.append(tweet)
    
    if len(tweets_filtered) == 0:
        # Means all the fetched tweets are unfiltered as of conflict
        print("No tweets to write database")
        
        last_tweet = tweets[len(tweets)-1]
        max_id = last_tweet['id'] - 1
    
    else:
        print("{} tweets written to db".format(len(tweets_filtered)))
        try:
            tweets_db.insert_many([reshape_tweet(tweet, hashtag_id) for tweet in tweets_filtered])
        except BulkWriteError as bwe:
            pprint(bwe.details)
            raise

        last_tweet = tweets_filtered[len(tweets_filtered)-1]
        max_id = last_tweet['id'] - 1

f.close()

Request 0
Number of tweets 91
91 tweets written to db
Request 1
Number of tweets 93
93 tweets written to db
Request 2
Number of tweets 90
90 tweets written to db
Request 3
Number of tweets 93
93 tweets written to db
Request 4
Number of tweets 98
Hashtag #cıldıracağım found in tweet 1253779493203951617
97 tweets written to db
Request 5
Number of tweets 77
77 tweets written to db
Request 6
Number of tweets 94
94 tweets written to db
Request 7
Number of tweets 96
96 tweets written to db
Request 8
Number of tweets 99
99 tweets written to db
Request 9
Number of tweets 100
100 tweets written to db
Request 10
Number of tweets 94
94 tweets written to db
Request 11
Number of tweets 92
92 tweets written to db
Request 12
Number of tweets 100
100 tweets written to db
Request 13
Number of tweets 100
100 tweets written to db
Request 14
Number of tweets 100
100 tweets written to db
Request 15
Number of tweets 100
Hashtag #işaş found in tweet 1253735382216253440
100 tweets written to db
Request 16
Num

https://twitter.com/sercanoksak/status/1246154630175875076

In [39]:
tweets_db.delete_many({'hashtag_id': '#gerçekfikrine'})

In [14]:
tweets_db.find_one({'id': 1250861860175450112})

In [44]:
max_id = 1251223630245232653 - 1

In [35]:
1252719203779567619 < 1251276936220901376

False

In [6]:
tweets_db = db.tweets_for_sentiment

In [9]:
tweet_search_continue = True
check_all_for_dup = True
hashtag_id = 'covid OR covid19 OR korona OR koronavirüs OR koronavırus OR kovid'
max_id = None
request_number = 0

In [12]:
f = open('conflicts.txt', 'a+')

while tweet_search_continue:
    
    print("Request {}".format(request_number))
    
    request_number += 1
    
    tweets = api.search(q=hashtag_id, count=100, lang="tr", max_id=max_id, tweet_mode="extended")
    
    if len(tweets) == 0:
        print("No tweets fetched")
        break
    
    tweets = [tweet._json for tweet in tweets]

    print("Number of tweets {}".format(len(tweets)))
    
    tweets_filtered = []
    for tweet in tweets:
        
        # Check if it is a RT, if RT do no include
        if tweet['full_text'].startswith('RT'):
            continue
        
        no_conflict = True
        
        if no_conflict:
            tweets_filtered.append(tweet)
    
    if len(tweets_filtered) == 0:
        # Means all the fetched tweets are unfiltered as of conflict
        print("No tweets to write database")
        
        last_tweet = tweets[len(tweets)-1]
        max_id = last_tweet['id'] - 1
    
    else:
        print("{} tweets written to db".format(len(tweets_filtered)))
        try:
            tweets_db.insert_many([reshape_tweet(tweet, hashtag_id) for tweet in tweets_filtered])
        except BulkWriteError as bwe:
            pprint(bwe.details)
            raise

        last_tweet = tweets_filtered[len(tweets_filtered)-1]
        max_id = last_tweet['id'] - 1

f.close()

Request 362
Number of tweets 100
12 tweets written to db
Request 363
Number of tweets 100
13 tweets written to db
Request 364
Number of tweets 100
15 tweets written to db
Request 365
Number of tweets 100
12 tweets written to db
Request 366
Number of tweets 100
20 tweets written to db
Request 367
Number of tweets 100
14 tweets written to db
Request 368
Number of tweets 100
13 tweets written to db
Request 369
Number of tweets 100
15 tweets written to db
Request 370
Number of tweets 100
8 tweets written to db
Request 371
Number of tweets 100
7 tweets written to db
Request 372
Number of tweets 100
6 tweets written to db
Request 373
Number of tweets 100
12 tweets written to db
Request 374
Number of tweets 100
9 tweets written to db
Request 375
Number of tweets 100
2 tweets written to db
Request 376
Number of tweets 100
4 tweets written to db
Request 377
Number of tweets 100
9 tweets written to db
Request 378
Number of tweets 100
12 tweets written to db
Request 379
Number of tweets 100
8 twe

Number of tweets 100
21 tweets written to db
Request 507
Number of tweets 100
23 tweets written to db
Request 508
Number of tweets 100
20 tweets written to db
Request 509
Number of tweets 100
20 tweets written to db
Request 510
Number of tweets 100
26 tweets written to db
Request 511
Number of tweets 100
21 tweets written to db
Request 512
Number of tweets 100
23 tweets written to db
Request 513
Number of tweets 100
29 tweets written to db
Request 514
Number of tweets 100
17 tweets written to db
Request 515
Number of tweets 100
16 tweets written to db
Request 516
Number of tweets 100
24 tweets written to db
Request 517
Number of tweets 100
17 tweets written to db
Request 518
Number of tweets 99
21 tweets written to db
Request 519
Number of tweets 100
27 tweets written to db
Request 520
Number of tweets 100
23 tweets written to db
Request 521
Number of tweets 100
19 tweets written to db
Request 522
Number of tweets 100
20 tweets written to db
Request 523
Number of tweets 100
18 tweets w

RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]